https://huggingface.co/blog/fine-tune-vit

https://huggingface.co/google/vit-base-patch16-224-in21k

https://colab.research.google.com/drive/1cGMg-aMz0FvN2FQarYLqDpdhVAuh9zed#scrollTo=ZXQfSacDNj-5

In [ ]:
!pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
 !kaggle datasets download -d anshtanwar/jellyfish-types

Dataset URL: https://www.kaggle.com/datasets/anshtanwar/jellyfish-types
License(s): Attribution 4.0 International (CC BY 4.0)
 67% 17.0M/25.5M [00:00<00:00, 44.1MB/s]
100% 25.5M/25.5M [00:00<00:00, 56.5MB/s]


In [ ]:
!unzip jellyfish-types.zip

Archive:  jellyfish-types.zip
  inflating: Moon_jellyfish/01.jpg   
  inflating: Moon_jellyfish/02.jpg   
  inflating: Moon_jellyfish/03.jpg   
  inflating: Moon_jellyfish/04.jpg   
  inflating: Moon_jellyfish/05.jpg   
  inflating: Moon_jellyfish/06.jpg   
  inflating: Moon_jellyfish/08.jpg   
  inflating: Moon_jellyfish/10.jpg   
  inflating: Moon_jellyfish/12.jpg   
  inflating: Moon_jellyfish/13.jpg   
  inflating: Moon_jellyfish/14.jpg   
  inflating: Moon_jellyfish/16.jpg   
  inflating: Moon_jellyfish/17.jpg   
  inflating: Moon_jellyfish/18.jpg   
  inflating: Moon_jellyfish/21.jpg   
  inflating: Moon_jellyfish/22.jpg   
  inflating: Moon_jellyfish/24.jpg   
  inflating: Moon_jellyfish/25.jpg   
  inflating: Moon_jellyfish/26.jpg   
  inflating: Moon_jellyfish/28.jpg   
  inflating: Moon_jellyfish/29.JPG   
  inflating: Moon_jellyfish/30.jpg   
  inflating: Moon_jellyfish/31.jpg   
  inflating: Moon_jellyfish/32.jpg   
  inflating: Moon_jellyfish/34.jpg   
  inflating: Moon_je

### ++ image

In [ ]:
dir_base = "./"
dir_moon = dir_base + "Moon_jellyfish"
dir_barrel = dir_base + "barrel_jellyfish"
dir_blue = dir_base + "blue_jellyfish"
dir_compass = dir_base + "compass_jellyfish"
dir_lions_mane = dir_base + "lions_mane_jellyfish"
dir_mauve_stinger = dir_base + "mauve_stinger_jellyfish"

In [ ]:
import cv2
import os
from PIL import Image

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            # Convert BGR to RGB
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # Resize the image
            img = cv2.resize(img, (224, 224))
            # Convert to PIL Image
            img = Image.fromarray(img)
            images.append(img)
    return images

In [ ]:
moon_images = load_images_from_folder(dir_moon)
barrel_images = load_images_from_folder(dir_barrel)
blue_images = load_images_from_folder(dir_blue)
compass_images = load_images_from_folder(dir_compass)
lions_mane_images = load_images_from_folder(dir_lions_mane)
mauve_stinger_images = load_images_from_folder(dir_mauve_stinger)

### ++ labels

In [ ]:
moon_labels = [0] * len(moon_images)
barrel_labels = [1] * len(barrel_images)
blue_labels = [2] * len(blue_images)
compass_labels = [3] * len(compass_images)
lions_mane_labels = [4] * len(lions_mane_images)
mauve_stinger_labels = [5] * len(mauve_stinger_images)

### ++ make dataset

In [ ]:

from datasets import Dataset
def make_dataset(image_list, label_list):
    data = {'image': image_list, 'label': label_list}
    dataset = Dataset.from_dict(data)
    return dataset

In [ ]:
moon_dataset = make_dataset(moon_images, moon_labels)
barrel_dataset = make_dataset(barrel_images, barrel_labels)
blue_dataset = make_dataset(blue_images, blue_labels)
compass_dataset = make_dataset(compass_images, compass_labels)
lions_mane_dataset = make_dataset(lions_mane_images, lions_mane_labels)
mauve_stinger_dataset = make_dataset(mauve_stinger_images, mauve_stinger_labels)

In [ ]:
from datasets import concatenate_datasets

concat_dataset = concatenate_datasets([
    moon_dataset,
    barrel_dataset,
    blue_dataset,
    compass_dataset,
    lions_mane_dataset,
    mauve_stinger_dataset,
])


### ++ datasets - class labels

In [ ]:
from datasets import ClassLabel

class_names = ['moon', 'barrel', 'blue', 'compass', 'lions_mane', 'mauve_stinger']
class_labels = ClassLabel(num_classes=len(class_names), names=class_names)

In [ ]:
ds_concat = concat_dataset.cast_column("label", class_labels)

Casting the dataset:   0%|          | 0/900 [00:00<?, ? examples/s]

### ++ ViT

In [ ]:
from transformers import ViTImageProcessor

model_name_or_path = 'google/vit-base-patch16-224-in21k'
processor = ViTImageProcessor.from_pretrained(model_name_or_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

### ++ preprocess

In [ ]:
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = processor([x for x in example_batch['image']], return_tensors='pt')

    # Don't forget to include the labels!
    # inputs['labels'] = example_batch['labels']
    inputs['label'] = example_batch['label']
    return inputs

In [ ]:
ds_concat_transform = ds_concat.with_transform(transform)
ds_shuffle = ds_concat_transform.shuffle(seed=42)

In [ ]:
ds_split = ds_shuffle.train_test_split(test_size=0.1)
ds_train = ds_split["train"].train_test_split(test_size=0.2)
ds_final_test = ds_split["test"]

### ++ trainer

In [ ]:
import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        # 'paths': torch.tensor([x['image'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)


In [ ]:
from transformers import ViTForImageClassification

labels = ['moon', 'barrel', 'blue', 'compass', 'lions_mane', 'mauve_stinger']

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit-base-jellyfish",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=10,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=ds_train["train"],
    eval_dataset=ds_train["test"],
    tokenizer=processor,
)


<ipython-input-36-0d2c5c0aa755>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


### ++ train

In [ ]:
train_results = trainer.train()

Step,Training Loss,Validation Loss,Accuracy
100,0.007300,0.001194,1.000000
200,0.075300,0.001639,1.000000
300,0.000700,0.000848,1.000000
400,0.000700,0.000737,1.000000


In [ ]:
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

***** train metrics *****
  epoch                    =        10.0
  total_flos               = 467678620GF
  train_loss               =      0.0307
  train_runtime            =  0:02:02.35
  train_samples_per_second =      52.962
  train_steps_per_second   =       3.351


### ++ eval

In [ ]:
metrics = trainer.evaluate(ds_final_test)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =       10.0
  eval_accuracy           =     0.9889
  eval_loss               =     0.0657
  eval_runtime            = 0:00:00.66
  eval_samples_per_second =    134.403
  eval_steps_per_second   =      17.92


### ++ zip results

In [ ]:
import zipfile

def zip_directory(source_dir, zip_file_name):
    with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        for root, dirs, files in os.walk(source_dir):
            for file in files:
                zip_file.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), source_dir))

source_dir = "./vit-base-jellyfish"
zip_file_name = "vit-base-jellyfish.zip"
zip_directory(source_dir, zip_file_name)

In [ ]:
from google.colab import files
files.download(zip_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# END